# **Comparación de modelos**

In [ ]:
import os
import joblib
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd
from sklearn.pipeline import Pipeline

In [ ]:
# Cargar el dataset de prueba
X_test = pd.read_csv('../data/processed/preprocessing.csv')
y_test = X_test.pop('stroke')  # Asumiendo que 'stroke' es la variable objetivo

# Diccionario para almacenar las rutas de los modelos
model_paths = {
    'Gradient Boosting': '../models/MGB/ModelGB.pkl',
    'LDA': '../models/lda/lda.pkl',
    'LightGBM': '../models/ligthgbm/lightGBM_stroke_model.pkl',
    'XGBoost': '../models/xgboost/xgboost_stroke_optimized_20250609_124848.pkl',
    'Extra Trees': '../models/extra_trees_model.pkl'
}

# Diccionario para almacenar resultados
results = {}

In [ ]:
# Evaluar cada modelo
for name, path in model_paths.items():
    try:
        # Cargar el modelo
        model = joblib.load(path)

        # Realizar predicciones
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]

        # Calcular métricas
        accuracy_test = accuracy_score(y_test, y_pred)
        precision_test = precision_score(y_test, y_pred)
        recall_test = recall_score(y_test, y_pred)
        f1_test = f1_score(y_test, y_pred)
        
        # Valores de entrenamiento y validación específicos para cada modelo
        if name == 'XGBoost':
            accuracy_train = 0.8524  # Del archivo xgboost.ipynb
            accuracy_valid = 0.8000  # Valor aproximado basado en las métricas de validación
        elif name == 'LightGBM':
            accuracy_train = 0.7307  # Balanced Accuracy del archivo lightgbm.ipynb
            accuracy_valid = 0.7756  # Test Balanced Accuracy del archivo lightgbm.ipynb
        elif name == 'Gradient Boosting':
            accuracy_train = 0.8177  # Del archivo de evaluación
            accuracy_valid = 0.7800  # Valor aproximado basado en las métricas
        elif name == 'LDA':
            accuracy_train = 0.813504  # Del archivo lda.ipynb (versión con umbral óptimo)
            accuracy_valid = 0.855422  # Del archivo lda.ipynb (versión con umbral óptimo)
        else:
            # Valores por defecto para otros modelos
            accuracy_train = accuracy_test
            accuracy_valid = accuracy_test
        
        diferencia = abs(accuracy_train - accuracy_valid)
        
        results[name] = {
            'Modelo': name,
            'Balanceador': 'SMOTE',  # Asumiendo que todos usan SMOTE
            'Accuracy Train': accuracy_train,
            'Accuracy Valid': accuracy_valid,
            'Accuracy Test': accuracy_test,
            'Precision Test': precision_test,
            'Recall Test': recall_test,
            'F1 Test': f1_test,
            'Diferencia abs': diferencia,
            'Tipo de ajuste': 'Buen ajuste' if diferencia < 0.01 and accuracy_valid > 0.7 else ('Overfitting' if diferencia > 0.05 else 'Underfitting'),
            'Umbral elegido': 0.5  # Umbral por defecto
        }
    except Exception as e:
        print(f"Error al evaluar {name}: {str(e)}")

Error al evaluar LDA: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- smoking_status_Unknown
- work_type_Govt_job
- work_type_Never_worked

Error al evaluar LightGBM: 'dict' object has no attribute 'predict'


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:24:29] WARNING: /workspace/src/collective/../data/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


In [ ]:
# Crear DataFrame con las columnas especificadas
cols = ['Ranking', 'Modelo', 'Balanceador', 'Accuracy Train', 'Accuracy Valid', 'Accuracy Test', 
        'Precision Test', 'Recall Test', 'F1 Test', 'Diferencia abs', 'Tipo de ajuste', 'Umbral elegido']

# Convertir resultados a DataFrame
df_resumen = pd.DataFrame.from_dict(results, orient='index')

# Calcular un score compuesto basado en múltiples métricas
df_resumen['Score Compuesto'] = (
    df_resumen['F1 Test'] * 0.3 +           # F1 Score tiene un peso del 30%
    df_resumen['Accuracy Test'] * 0.2 +     # Accuracy tiene un peso del 20%
    df_resumen['Precision Test'] * 0.2 +    # Precision tiene un peso del 20%
    df_resumen['Recall Test'] * 0.2 +       # Recall tiene un peso del 20%
    (1 - df_resumen['Diferencia abs']) * 0.1  # Penalización por diferencia entre train y valid (10%)
)

# Agregar ranking basado en el score compuesto
df_resumen['Ranking'] = df_resumen['Score Compuesto'].rank(ascending=False).astype(int)

# Reordenar columnas
df_resumen = df_resumen[cols]

# Agregar columnas extra solicitadas
df_resumen['De 10 enviados, ¿cuántos realmente tienen ACV? (Precision)'] = (df_resumen['Precision Test'] * 10).round(1)
df_resumen['De 10 enfermos reales, ¿cuántos detecta? (Recall)'] = (df_resumen['Recall Test'] * 10).round(1)
df_resumen['¿Buen ajuste?'] = df_resumen['Tipo de ajuste'].apply(lambda x: 'Sí' if x == 'Buen ajuste' else 'No')

# Mostrar resultados
display(df_resumen)

In [ ]:
# Función para extraer y generar el pipeline
def extract_pipeline(model):
    steps = []

    # Si el modelo ya es un pipeline, extraer sus pasos
    if isinstance(model, Pipeline):
        return model

    # Si no es un pipeline, crear uno nuevo con el modelo
    if hasattr(model, 'preprocessor_') and hasattr(model, 'estimator_'):
        # Para modelos que tienen preprocessor y estimator separados
        steps.extend([
            ('preprocessor', model.preprocessor_),
            ('estimator', model.estimator_)
        ])
    else:
        # Para modelos simples
        steps.append(('estimator', model))

    return Pipeline(steps)

    # Extraer/generar el pipeline del mejor modelo
best_pipeline = extract_pipeline(best_model)

# Crear el nombre del archivo para el pipeline
pipeline_filename = f'best_pipeline_{best_model_name.lower().replace(" ", "_")}.pkl'

# Define the path to the models directory
models_dir = '../models'
# Create the models directory if it doesn't exist
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

pipeline_path = os.path.join(models_dir, pipeline_filename)

# Guardar el pipeline
joblib.dump(best_pipeline, pipeline_path)

# Mostrar información del pipeline
print("\nPipeline del mejor modelo:")
for step_name, step in best_pipeline.steps:
    print(f"- {step_name}: {type(step).__name__}")

print(f"\nPipeline guardado en: {pipeline_path}")


Resultados de la comparación:
                   accuracy  precision  recall      f1  roc_auc
Gradient Boosting    0.8177     0.1693  0.6815  0.2713   0.8572
XGBoost              0.7792     0.1592  0.8024  0.2657   0.8586

Mejor modelo: Gradient Boosting
